In [61]:
import json
import csv
import os

In [62]:
def empty_csv_dict():
    return {
        'model': '',
        'filename': '',
        'model_params': '',
        'augmentation': None,
        'brightness': None,
        'precipitation': None,
        'surface': None,
        'weather': None,
        'ground_truth_objects': 0,
        'segmented_objects': 0,
        'matched_objects': 0,
        'precision': 0.0,
        'recall': 0.0,
        'f1': 0.0,
        'mean_iou': 0.0,
        'time': 0,
        'hardware': 'NVIDIA GeForce RTX 3060M, AMD Ryzen 7 5800H, 32 GB RAM'
    }

def segm_dict2csv_dict(segm_dict):
    csv_dict = empty_csv_dict()

    weather_options = [
        ["bright", "dark", "night"],
        ["none", "rainfall", "snowfall", "fog"],
        ['none', 'puddles', 'snow'],
        ["none", "sunny", "cloudy"]
    ]
    weather_fields = ['brightness', 'precipitation', 'surface', 'weather']

    # get weather options
    for k, options in zip(weather_fields, weather_options):
        for option in options:
            if option in segm_dict['weather']:
                csv_dict[k] = option
    
    # get model
    csv_dict['model'] = ' '.join(segm_dict['model'])
    # print(csv_dict['model'], 'fastsam' in csv_dict['model'] or 'yolo' in csv_dict['model'])
    if 'fastsam' in csv_dict['model'] or 'yolo' in csv_dict['model']:
        csv_dict['model'] = csv_dict['model'].replace('sam2.1 ', '')

    if 'model_params' in segm_dict:
        csv_dict['model_params'] = segm_dict['model_params']

    csv_dict['filename'] = segm_dict['filename']
    csv_dict['time'] = segm_dict['time']
    
    # get numerical fields
    for k in ['ground_truth_objects', 'segmented_objects', 'matched_objects', 'precision', 'recall', 'f1', 'mean_iou']:
        if k in segm_dict:
            csv_dict[k] = segm_dict[k]
        else:
            print(f"Warning: key={k} not in segm_dict({segm_dict['model']}, {segm_dict['filename']})")
            csv_dict[k] = 0

    return csv_dict

In [106]:
results_path = r"C:\Users\gtraw\OneDrive\Pulpit\UM sem. 2\ProjektBadawczy\apps\evaluation\results\all_11_05"

In [ ]:
# load data from jsons

data = []
for f in os.listdir(results_path):
    with open(os.path.join(results_path, f), 'r') as json_file:
        json_data = json.load(json_file)
        if 'filename' not in json_data:
            json_data['filename'] = '.'.join(f.split('.')[:-1])
        if '_5_' in f:
            json_data['model_params'] = 'conf=0.5'
        elif '_1_' in f:
            json_data['model_params'] = 'conf=0.1'
        elif '_01_' in f:
            json_data['model_params'] = 'conf=0.01'
        data.append(json_data)

In [108]:
data[123]

{'weather': ['dark', 'sunny'],
 'model': ['sam2.1', 'fastsam_x'],
 'time': 2.011305570602417,
 'ground_truth_objects': 13,
 'segmented_objects': 41,
 'matched_objects': 13,
 'precision': 0.3170731707317073,
 'recall': 1.0,
 'f1': 0.48148148148148145,
 'mean_iou': 0.759907457315857,
 'filename': 'weti1_2024_11_05_12_31_m'}

In [ ]:
# transform jsons

csv_dicts = [segm_dict2csv_dict(d) for d in data]

In [110]:
csv_dicts[111]

{'model': 'fastsam_x',
 'filename': 'weti1_2024_11_05_11_16_m',
 'model_params': '',
 'augmentation': None,
 'brightness': 'dark',
 'precipitation': None,
 'surface': None,
 'weather': 'sunny',
 'ground_truth_objects': 13,
 'segmented_objects': 37,
 'matched_objects': 13,
 'precision': 0.35135135135135137,
 'recall': 1.0,
 'f1': 0.52,
 'mean_iou': 0.7469452291227885,
 'time': 0.8052759170532227,
 'hardware': 'NVIDIA GeForce RTX 3060M, AMD Ryzen 7 5800H, 32 GB RAM'}

In [ ]:
# save to csv file

keys = csv_dicts[0].keys()
output_path = r"C:\Users\gtraw\OneDrive\Pulpit\UM sem. 2\ProjektBadawczy\apps\evaluation\csvs\2024_11_05.csv"

with open(output_path, 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(csv_dicts)

In [112]:
import pandas as pd

df = pd.read_csv(output_path)
print(df.describe())
print(df.head)

       augmentation  precipitation  surface  ground_truth_objects  \
count           0.0            0.0      0.0           1100.000000   
mean            NaN            NaN      NaN             24.720000   
std             NaN            NaN      NaN             14.599842   
min             NaN            NaN      NaN              7.000000   
25%             NaN            NaN      NaN             12.000000   
50%             NaN            NaN      NaN             18.000000   
75%             NaN            NaN      NaN             43.000000   
max             NaN            NaN      NaN             44.000000   

       segmented_objects  matched_objects    precision       recall  \
count        1100.000000      1100.000000  1100.000000  1100.000000   
mean           47.400909        16.006364     0.603732     0.645122   
std            66.447323        12.081576     0.315917     0.253059   
min             0.000000         0.000000     0.000000     0.000000   
25%            11.00000

In [113]:
for i, m in enumerate(df['model'].unique()):
    print(f"{i+1:<3} {m}")

1   fastsam_s
2   fastsam_x
3   sam2.1 base+
4   sam2.1 large
5   sam2.1 small
6   sam2.1 tiny
7   yolo11l
8   yolo11m
9   yolo11n
10  yolo11s
11  yolo11x
